In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu")

True
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_DIR = "./kmbert_epoch3"
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR).to(device).eval()

id2label = model.config.id2label
label2id = model.config.label2id
print("num_labels:", len(id2label))

num_labels: 583


In [3]:
import pandas as pd

labels_csv = r"C:\Users\Jang8\OneDrive\바탕 화면\졸작\데이터\mediq_labels_final.csv"
scope_csv  = r"C:\Users\Jang8\OneDrive\바탕 화면\졸작\데이터\mediq_scope_rules.csv"

df_labels = pd.read_csv(labels_csv)
df_scope  = pd.read_csv(scope_csv)

print(df_labels.shape, df_scope.shape)

(583, 5) (106, 2)


In [4]:
import re
import torch
import torch.nn.functional as F

# -----------------------------
# 0) 유틸: 컬럼 자동 찾기
# -----------------------------
def _pick_col(cols, candidates):
    cols_l = {c.lower(): c for c in cols}
    for cand in candidates:
        cand_l = cand.lower()
        if cand_l in cols_l:
            return cols_l[cand_l]
    # 부분 포함 매칭
    for c in cols:
        cl = c.lower()
        for cand in candidates:
            if cand.lower() in cl:
                return c
    return None

def _to_str(x):
    return "" if x is None else str(x)

# -----------------------------
# 1) df_labels에서 라벨->메타 매핑 만들기
#    (진료과/위험도/설명 등)
# -----------------------------
label_col = _pick_col(df_labels.columns, ["label", "standard_symptom", "표준증상", "증상", "disease", "병명"])
dept_col  = _pick_col(df_labels.columns, ["dept", "department", "진료과", "진료"])
sev_col   = _pick_col(df_labels.columns, ["severity", "risk", "위험도", "중증도"])
desc_col  = _pick_col(df_labels.columns, ["desc", "description", "설명", "코멘트", "comment", "정의"])

if label_col is None:
    raise ValueError(f"df_labels에서 라벨 컬럼을 못 찾음. columns={df_labels.columns.tolist()}")

LABEL2DEPT = {}
LABEL2SEV  = {}
LABEL2DESC = {}

for _, row in df_labels.iterrows():
    lbl = _to_str(row.get(label_col)).strip()
    if not lbl:
        continue
    if dept_col:
        LABEL2DEPT[lbl] = _to_str(row.get(dept_col)).strip()
    if sev_col:
        LABEL2SEV[lbl] = _to_str(row.get(sev_col)).strip()
    if desc_col:
        LABEL2DESC[lbl] = _to_str(row.get(desc_col)).strip()

# 기본값
DEFAULT_DEPT = "내과"
DEFAULT_SEV  = "일반"

# -----------------------------
# 2) df_scope에서 룰 리소스 구성
#    - red flag 키워드
#    - non-target(배제) 라벨/키워드
#    - cancer/rare 억제 키워드
# -----------------------------
# df_scope는 팀이 만든 구조가 다양할 수 있어서,
# "category / keyword / label / type" 같은 이름들을 최대한 자동으로 찾아 쓴다.
cat_col   = _pick_col(df_scope.columns, ["category", "type", "rule_type", "분류", "구분"])
kw_col    = _pick_col(df_scope.columns, ["keyword", "kw", "pattern", "키워드", "표현"])
label2_col = _pick_col(df_scope.columns, ["label", "target_label", "라벨", "표준증상"])
val_col   = _pick_col(df_scope.columns, ["value", "val", "enabled", "flag"])

# category 후보들(너가 말한 스코프 재정의 기준 반영)
RED_FLAG_CATS = {"red_flag", "emergency", "out_of_scope_emergency", "응급", "레드플래그", "out_of_scope"}
EXCLUDE_CATS  = {"exclude_label", "non_target_label", "non_target", "배제", "중증배제", "rare_exclude", "cancer_exclude"}
CANCER_CATS   = {"cancer", "tumor", "암", "종양", "rare", "희귀"}

red_flag_keywords = set()
exclude_labels = set()
cancer_labels = set()

if cat_col and (kw_col or label2_col):
    for _, r in df_scope.iterrows():
        cat = _to_str(r.get(cat_col)).strip().lower()
        kw  = _to_str(r.get(kw_col)).strip() if kw_col else ""
        lbl = _to_str(r.get(label2_col)).strip() if label2_col else ""

        # red flag
        if cat in RED_FLAG_CATS:
            if kw:
                red_flag_keywords.add(kw)

        # exclude labels (스코프 밖: 암/희귀/초응급 등)
        if cat in EXCLUDE_CATS:
            if lbl:
                exclude_labels.add(lbl)
            elif kw:
                # 키워드로도 배제하고 싶으면 kw를 라벨처럼 취급할 수 있음
                exclude_labels.add(kw)

        # cancer/rare labels
        if cat in CANCER_CATS:
            if lbl:
                cancer_labels.add(lbl)

# label 리스트 기반으로 암/희귀 후보를 "자동 탐지" (CSV에 따로 없더라도)
# - 너무 공격적으로 걸면 문제라, '암'/'종양'/'림프종' 등만 기본 후보로 탐지
auto_cancer = set()
for lbl in id2label.values():
    if ("암" in lbl and "양성" not in lbl and "선종" not in lbl) or ("종양" in lbl) or ("림프종" in lbl):
        auto_cancer.add(lbl)
cancer_labels |= auto_cancer

# 억제 키워드(UT 피드백 반영)
CANCER_TRIGGER_KEYWORDS = [
    "덩어리", "혹", "멍울", "종괴",
    "체중 빠져", "살이 빠져", "급격히 살", "식욕 저하",
    "혈변", "검은 변", "피 섞인 변", "피가 섞여",
    "혈뇨", "피 오줌", "피를 토", "토혈",
    "오래 지속", "몇 달째", "몇개월째", "계속 아프", "점점 심해",
]

# -----------------------------
# 3) 0단계 Safety Check (레드플래그 즉시 종료)
# -----------------------------
# (너가 이전에 갖고 있던 out_of_scope 키워드 csv도 여기에 같이 반영됨)
DEFAULT_RED_FLAGS = [
    "숨이 안 쉬어", "호흡곤란", "질식", "의식 잃", "기절", "쓰러",
    "마비", "말이 안 나", "입이 돌아", "시야가 안", "갑자기 안 보",
    "가슴이 쥐어짜", "피를 토", "토혈", "대량 출혈", "경련",
]
for k in DEFAULT_RED_FLAGS:
    red_flag_keywords.add(k)

def safety_check(text: str):
    t = text.strip()
    hits = [kw for kw in red_flag_keywords if kw and (kw in t)]
    if hits:
        return {
            "ok": False,
            "hits": hits[:10],
            "message": "⚠️ 현재 입력에는 응급 신호가 포함될 수 있어요. 즉시 119 또는 응급실 방문을 권장합니다. (서비스 범위 밖)"
        }
    return {"ok": True, "hits": [], "message": ""}

# -----------------------------
# 4) 모델 추론: top-k
# -----------------------------
@torch.no_grad()
def predict_topk(text: str, k=5):
    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding=False
    )
    enc = {k: v.to(device) for k, v in enc.items()}
    logits = model(**enc).logits[0]
    probs = F.softmax(logits, dim=-1)
    top = torch.topk(probs, k=k)
    out = []
    for idx, p in zip(top.indices.tolist(), top.values.tolist()):
        out.append((id2label[idx], float(p)))
    return out

# -----------------------------
# 5) 스코프/억제/출력정책
#    - 암/희귀는 "트리거 없으면" 하단에만 숨기거나 제거
#    - 배제 라벨(exclude_labels)은 최종 선택에서 제외하고 안내로만
# -----------------------------
def _has_any_keyword(text, kws):
    t = text.strip()
    return any((kw in t) for kw in kws)

def apply_scope_and_suppression(text: str, topk):
    reasons = []
    t = text.strip()

    # 1) exclude label이 1위면, 다음 후보로 내려보냄(최종 선택에서 제외)
    filtered = [(lbl, p) for (lbl, p) in topk if lbl not in exclude_labels]
    if len(filtered) < len(topk):
        reasons.append("exclude_label_filtered")

    # 2) 암/희귀 억제: 트리거 없으면 메인 후보에서 제외하고 extra로 분리
    cancer_trig = _has_any_keyword(t, CANCER_TRIGGER_KEYWORDS)
    extras = []
    main = []

    for lbl, p in filtered:
        if lbl in cancer_labels and not cancer_trig:
            extras.append((lbl, p))
        else:
            main.append((lbl, p))

    if extras and not cancer_trig:
        reasons.append("cancer_suppressed_to_extra")

    # main이 비었으면(전부 억제/배제), 그때만 억제 해제해서 하나 살림
    if not main:
        main = filtered[:]
        reasons.append("fallback_unsuppress_due_to_empty")

    final_label, final_prob = main[0]
    return {
        "final": (final_label, final_prob),
        "main_topk": main[:5],
        "extra_risk": extras[:5],  # UI에서 "추가 위험 후보(낮은 가능성)" 섹션으로
        "reasons": reasons,
        "cancer_trigger": cancer_trig,
    }

# -----------------------------
# 6) UI payload 빌더 (너가 보낸 결과 화면 구조용)
# -----------------------------
def build_ui_payload(user_text: str, k=5):
    # 0) safety
    s = safety_check(user_text)
    if not s["ok"]:
        return {
            "status": "blocked",
            "warning": s["message"],
            "hits": s["hits"],
            "user_text": user_text
        }

    # 1) topk
    topk = predict_topk(user_text, k=max(k, 5))
    # 2) scope/suppress
    ss = apply_scope_and_suppression(user_text, topk)

    final_label, final_prob = ss["final"]
    dept = LABEL2DEPT.get(final_label, DEFAULT_DEPT) or DEFAULT_DEPT
    sev  = LABEL2SEV.get(final_label, DEFAULT_SEV) or DEFAULT_SEV
    desc = LABEL2DESC.get(final_label, "") or ""

    # 3) 메시지(간단)
    disclaimer = "아래 결과는 참고용이며 실제 진단이 아닙니다. 증상이 지속/악화되면 의료진 진료를 권장합니다."
    if sev in ["응급", "긴급"]:
        warn = "⚠️ 위험 신호 가능성이 있어요. 즉시 응급실 방문/119를 권장합니다."
    elif sev in ["주의"]:
        warn = "주의가 필요할 수 있어요. 가능한 빠른 시일 내 진료를 권장합니다."
    else:
        warn = "대부분은 경과 관찰이 가능하지만, 지속되면 진료를 권장합니다."

    return {
        "status": "ok",
        "user_text": user_text,
        "final": {"label": final_label, "prob": final_prob, "dept": dept, "severity": sev},
        "main_top5": [{"label": l, "prob": p} for l, p in ss["main_topk"]],
        "extra_risk": [{"label": l, "prob": p} for l, p in ss["extra_risk"]],
        "desc": desc,
        "disclaimer": disclaimer,
        "warning": warn,
        "debug": {
            "reasons": ss["reasons"],
            "cancer_trigger": ss["cancer_trigger"],
        }
    }

# -----------------------------
# 7) 빠른 성능/출력 테스트
# -----------------------------
tests = [
    "아랫배가 아파요",
    "임신 8주차인데 아랫배가 아파요",
    "며칠째 기침하고 숨쉬기 힘들어요",
    "혈변이 나와요",
    "눈이 너무 뻑뻑하고 따가워요",
    "신물이 올라오고 속이 쓰려요",
    "가슴이 쥐어짜듯 아파요",   # safety에 걸리게 설계됨
]

for t in tests:
    out = build_ui_payload(t, k=5)
    print("="*70)
    print("입력:", t)
    print("status:", out["status"])
    if out["status"] == "blocked":
        print("경고:", out["warning"])
        continue
    print("final:", out["final"])
    print("top5:", [(x["label"], round(x["prob"], 3)) for x in out["main_top5"]])
    if out["extra_risk"]:
        print("extra_risk:", [(x["label"], round(x["prob"], 3)) for x in out["extra_risk"]])
    print("debug:", out["debug"])

입력: 아랫배가 아파요
status: ok
final: {'label': '골반 염증성 질환', 'prob': 0.05794139578938484, 'dept': '산부인과', 'severity': '주의'}
top5: [('골반 염증성 질환', 0.058), ('월경전 증후군', 0.056), ('난소난관염', 0.042), ('난소, 난관 이상', 0.025)]
extra_risk: [('난소의 양성 종양', 0.547)]
debug: {'reasons': ['cancer_suppressed_to_extra'], 'cancer_trigger': False}
입력: 임신 8주차인데 아랫배가 아파요
status: ok
final: {'label': '유산', 'prob': 0.7873330116271973, 'dept': '산부인과', 'severity': '주의'}
top5: [('유산', 0.787), ('조기 진통', 0.037), ('습관성 유산', 0.032), ('조산', 0.028), ('자궁 내 성장 지연', 0.009)]
debug: {'reasons': [], 'cancer_trigger': False}
입력: 며칠째 기침하고 숨쉬기 힘들어요
status: ok
final: {'label': '중증 급성 호흡기 증후군', 'prob': 0.13765732944011688, 'dept': '내과', 'severity': '일반'}
top5: [('중증 급성 호흡기 증후군', 0.138), ('심낭염', 0.132), ('기도 폐쇄', 0.115), ('지역사회성 폐렴', 0.047), ('천식', 0.043)]
debug: {'reasons': [], 'cancer_trigger': False}
입력: 혈변이 나와요
status: blocked
경고: ⚠️ 현재 입력에는 응급 신호가 포함될 수 있어요. 즉시 119 또는 응급실 방문을 권장합니다. (서비스 범위 밖)
입력: 눈이 너무 뻑뻑하고 따가워요
status: ok
final: {'labe

In [23]:
import pandas as pd

path = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\MediQ_UI_Tokens__v1_.csv"
df_tokens = pd.read_csv(path, encoding="utf-8-sig")

df_tokens.head()

,level,code,ko_label,parent_code,notes_examples
0,BODY_PART,BP_HEAD_FACE,머리/얼굴,NaN,2D 모델 영역: 머리/얼굴
1,BODY_PART,BP_NECK_CHEST,목/가슴,NaN,2D 모델 영역: 목/가슴
2,BODY_PART,BP_ABDOMEN_PELVIS,복부/골반,NaN,2D 모델 영역: 복부/골반
3,BODY_PART,BP_EXTREMITIES,팔/다리,NaN,2D 모델 영역: 팔/다리
4,BODY_PART,BP_SYSTEMIC,전신/기타,NaN,피부/신경/전신 증상/등·허리 포함


In [24]:
import pandas as pd

# df_tokens 가 이미 로드되어 있다고 가정
# df_tokens.columns: level, code, ko_label, parent_code, notes_examples

TOKENS = df_tokens.copy()

# code 존재 여부 빠른 검증용 set
TOKEN_CODES = set(TOKENS["code"].astype(str))

# code -> 한글 라벨
CODE2KO = dict(zip(TOKENS["code"].astype(str), TOKENS["ko_label"].astype(str)))

# 한글 라벨 -> code (동명이인 라벨이 있을 수 있어서 list로)
from collections import defaultdict
KO2CODES = defaultdict(list)
for c, k in zip(TOKENS["code"].astype(str), TOKENS["ko_label"].astype(str)):
    KO2CODES[k].append(c)

def is_valid_code(code: str) -> bool:
    return (code is not None) and (str(code) in TOKEN_CODES)

def code_to_ko(code: str) -> str:
    return CODE2KO.get(str(code), str(code))

In [25]:
def build_symptom_input(ui_state: dict) -> dict:
    """
    ui_state 예시(너희 앱/프론트에서 넘어오는 값):
      {
        "body_part": "BP_HEAD_FACE",
        "region": "REG_HEAD",
        "sub_region": "SUB_HEAD_TEMPORAL",
        "micro_location": "MICRO_TEMPLES_VASCULAR",
        "feeling": "THROBBING",          # 느낌 토큰 (아직 없으면 문자열로)
        "intensity": 7,                 # 0~10
        "onset": "TODAY",               # 시작시점 토큰 (아직 없으면 문자열로)
        "associated": ["PHOTOPHOBIA"],  # 동반증상 토큰 리스트
        "free_text": "관자놀이가 지끈거려요"
      }
    """

    def _get_code(key):
        v = ui_state.get(key)
        return str(v) if v is not None else None

    symptom = {
        # 위치 토큰(이번에 만든 csv 기반)
        "body_part": _get_code("body_part"),
        "region": _get_code("region"),
        "sub_region": _get_code("sub_region"),
        "micro_location": _get_code("micro_location"),

        # 아래는 아직 토큰 csv를 안 만들었으니 일단 문자열/숫자로 받자
        "feeling": ui_state.get("feeling"),             # 예: "THROBBING" / "PRESSING" ...
        "intensity": ui_state.get("intensity"),         # 예: 0~10
        "onset": ui_state.get("onset"),                 # 예: "JUST_NOW" / "TODAY" / "2_3_DAYS" ...
        "associated": ui_state.get("associated", []) or [],

        "free_text": (ui_state.get("free_text") or "").strip()
    }

    # --- 최소 검증 (위치 토큰) ---
    # body_part/region/sub_region/micro_location는 선택형이라면 code가 유효해야 함
    for k in ["body_part", "region", "sub_region", "micro_location"]:
        v = symptom.get(k)
        if v is not None and v != "" and (not is_valid_code(v)):
            raise ValueError(f"[TokenError] {k}={v} 는 df_tokens에 없는 code입니다.")

    # intensity 표준화(없으면 None)
    if symptom["intensity"] is not None:
        try:
            symptom["intensity"] = int(symptom["intensity"])
        except:
            raise ValueError(f"[ValueError] intensity는 정수(0~10)로 와야 합니다. 지금 값={symptom['intensity']}")

    return symptom

In [26]:
def build_model_text(symptom: dict, include_location_hint: bool = True) -> str:
    """
    KM-BERT에 넣을 텍스트 생성.
    - free_text가 있으면 그게 메인
    - 위치/느낌/강도/시작시점은 보조 힌트로 짧게 추가(옵션)
    """

    parts = []

    free_text = (symptom.get("free_text") or "").strip()
    if free_text:
        parts.append(free_text)

    if include_location_hint:
        # code를 한글로 변환해서 짧게 힌트 제공
        loc = []
        for k in ["body_part", "region", "sub_region", "micro_location"]:
            code = symptom.get(k)
            if code:
                loc.append(code_to_ko(code))
        if loc:
            parts.append(" / ".join(loc))

        if symptom.get("feeling"):
            parts.append(f"느낌:{symptom['feeling']}")
        if symptom.get("intensity") is not None:
            parts.append(f"강도:{symptom['intensity']}/10")
        if symptom.get("onset"):
            parts.append(f"시작:{symptom['onset']}")
        if symptom.get("associated"):
            parts.append("동반:" + ",".join([str(x) for x in symptom["associated"]]))

    text = " | ".join([p for p in parts if p and str(p).strip()])
    return text.strip()

In [27]:
ui_state_example = {
    "body_part": "BP_HEAD_FACE",
    "region": "REG_HEAD",
    "sub_region": "SUB_HEAD_TEMPORAL",
    "micro_location": "MICRO_TEMPLES_VASCULAR",
    "feeling": "THROBBING",
    "intensity": 7,
    "onset": "TODAY",
    "associated": ["PHOTOPHOBIA"],
    "free_text": "관자놀이가 지끈거리고 아파요"
}

sym = build_symptom_input(ui_state_example)
print(sym)

model_text = build_model_text(sym)
print("\n[MODEL TEXT]\n", model_text)

{'body_part': 'BP_HEAD_FACE', 'region': 'REG_HEAD', 'sub_region': 'SUB_HEAD_TEMPORAL', 'micro_location': 'MICRO_TEMPLES_VASCULAR', 'feeling': 'THROBBING', 'intensity': 7, 'onset': 'TODAY', 'associated': ['PHOTOPHOBIA'], 'free_text': '관자놀이가 지끈거리고 아파요'}

[MODEL TEXT]
 관자놀이가 지끈거리고 아파요 | 머리/얼굴 / 두부(Head) / 옆머리/관자 / 관자놀이-혈관 부위 | 느낌:THROBBING | 강도:7/10 | 시작:TODAY | 동반:PHOTOPHOBIA
